# Demo notebook: Math + Econ Reasoning Portfolio

This notebook demonstrates:
- CPI targeting function and the bisection solution
- Contract task: utility target vs. c_high (and the solved value)
- Credit VaR: analytic value + Monte Carlo sanity-check distribution

Run after installing dev deps:
```bash
pip install -e ".[dev]"
```


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

from econ_math_portfolio.models.cpi_target_discount import CpiParams, cpi, solve_t
from econ_math_portfolio.models.contract_stochastic_income import ContractParams, lifetime_utility, solve_c_high
from econ_math_portfolio.models.credit_var_quantile import CreditParams, var_analytic, var_mc


In [ ]:
# CPI(t) curve + target
p = CpiParams()
ts = np.linspace(0, 1, 400)
vals = [cpi(float(t), p) for t in ts]
t_star = solve_t(p)

plt.figure()
plt.plot(ts, vals)
plt.axhline(p.target_cpi)
plt.axvline(t_star)
plt.title("CPI(t) and target")
plt.xlabel("t")
plt.ylabel("CPI(t)")
plt.show()

t_star


In [ ]:
# Contract task: utility as a function of c_high + solved c_high
cp = ContractParams()
c_grid = np.linspace(0.5, 3.0, 400)
u_vals = [lifetime_utility(cp.delta, cp.c_low, float(c)) for c in c_grid]
c_star = solve_c_high(cp)

plt.figure()
plt.plot(c_grid, u_vals)
plt.axhline(cp.V0)
plt.axvline(c_star)
plt.title("Lifetime utility target vs c_high")
plt.xlabel("c_high")
plt.ylabel("V(c_high)")
plt.show()

c_star


In [ ]:
# Credit VaR: analytic + Monte Carlo sanity check (infinitely granular Vasicek)
pp = CreditParams()
analytic = var_analytic(pp)
mc_q = var_mc(pp, n_paths=200_000, seed=7)

# Show distribution of VaR estimates from multiple seeds
estimates = [var_mc(pp, n_paths=50_000, seed=s) for s in range(10, 60)]
plt.figure()
plt.hist(estimates, bins=20)
plt.axvline(analytic)
plt.title("MC VaR estimates (batch) vs analytic VaR")
plt.xlabel("VaR estimate")
plt.ylabel("count")
plt.show()

analytic, mc_q
